## Map area: San Francisco, California, United States (data downloaded from openstreetmap)


In [1]:
import xml.etree.cElementTree as ET
import pprint
from collections import defaultdict
import re

In [2]:
sf_data = "san-francisco_california.osm"
#sf_data = "sample_sf.osm"

## Create a sample data set 

In [20]:
# This code is adapted from the Udacity project instruction
#!/usr/bin/env python
# -*- coding: utf-8 -*-


OSM_FILE = "san-francisco_california.osm"  # Replace this with your osm file
SAMPLE_FILE = "sample_sf_1.osm"


### Define a get_element function to create a generator over the elements in the data file

In [21]:
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag
    Reference:
    http://stackoverflow.com/questions/231767/what-does-the-yield-keyword-do-in-python
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

In [22]:
with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every 10th top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % 10 == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

## Overview of the data structure of osm file

In [23]:
#define a function to look through the tags in the documenet
def count_tags(filename):

    result = defaultdict(int)
    for elem in get_element(sf_data):
        result[elem.tag] +=1
        elem.clear()
    return result


In [25]:
count_tags('san-francisco_california.osm')

defaultdict(int, {'node': 4466973, 'relation': 4424, 'way': 504247})

# 1 Problems encounted in the map

## 1.1 Explore tags on a sample set to identify basic format for tags

In [86]:
#explore a subset of uniques tags in the data file
unqiue_tags = set()

for i, elem in enumerate(get_element("sample_sf.osm", tags = ("tag",))):
    if i% 2000==0:
        unique_tags.add(elem.attrib["k"])
        
    elem.clear()

In [87]:
list(unique_tags)

['tiger:source',
 'maxspeed',
 'inside_atm',
 'name:nds-nl',
 'seamark:fog_signal:group',
 'destination:forward',
 'Asset',
 'cooperative',
 'smoking',
 'paloalto_ca:id',
 'name:gag',
 'outside_atm_operator',
 'Shape_Leng',
 'is_in',
 'building:height',
 'name:bat-smg',
 'seamark:type',
 'name:hsb',
 'name:ast',
 'created_by',
 'ref:blklot',
 'common',
 'restriction',
 'name:kn',
 'name:zu',
 'name:za',
 'fax',
 'tiger:LINEARID',
 'attribution',
 'name:ms',
 'name:uz',
 'name:kk',
 'AREAID',
 'tiger:cfcc',
 'redwood_city_ca:addr_id',
 'recycling:magazines',
 'tiger:county',
 'name:ki',
 'name:uk',
 'motor_vehicle',
 'name:ug',
 'ticker',
 'name:rw',
 'addr:street',
 'source:name',
 'name:mr',
 'level',
 'parking:condition:right:2',
 'seamark:radar_transponder:wavelength',
 'subway',
 'is_in:continent',
 'depository',
 'name:ks',
 'sidewalk',
 'disused',
 'recycling:paper',
 'nps:trail',
 'turn:lanes:forward',
 'name:ia',
 'name:lez',
 'gnis:state_id',
 'bicycle',
 'gnis:county_name',
 

### There are many different types of tag attributes. I'd like to further explore "addr:postcode", "phone", and  "addr:street" to see if the data is consistent.

## 1.2 Audit postcode 

In [88]:
postcode_output = set()
for elem in get_element(sf_data, tags = ("tag",)):
    if elem.get('k') == "addr:postcode":
        postcode_output.add(elem.get('v')) 
    elem.clear()

print postcode_output

set(['CA 94544', '94213', 'CA 94541', '94002-2121', 'CA 94546', '94121-3131', '94404', '94401', '94402', '94403', '94563', '94560', '94118', '94611', '94610', 'CA 94133', '94612', '94112', '94113', '94110', '94111', '94116', '94618', '94114', '94115', 'CA:94103', '90214', '94128', '94612-2202', '94127', '94579', '94124', '94123', '94122', '94121', '94129', '94577', '94606', '94607', '94605', '94602', '94603', '94601', '94044', '94801', '94118-4504', '94804', '94805', '94608', '94609', '94965', '94964', '1087', '94720-1076', 'CA 94404', '94118-1316', '94130', '94131', '94132', '94133', '94134', '94549-5506', '94038', '94970', '94030', '94507', '94501', '94037', '94502', '94587', '94580', '14123', '94025-1246', '94518', '94303', '94301', '94901', '95498', '94904', '94143', '94519', '94027', '94025', '94596', '94597', '9412', '94595', 'CA 94560', '515', 'CA 94080', '94598', '941234', '94619', 'ca', '94117', '94017', '94720', '94015', '94014', '94013', '94010', '25426', '94019', '94606-363

### In the above postcode set, we see that most postcode is in five-digit format. However, there are some other format, like "94121-3131", "1087" and "CA 94598", other than five-digit postcode. My next step is to write a function to audit postcode.

In [89]:
# define a function to audit postcode
def audit_postcode(postcode):
    p = re.findall('([0-9]{5})-?', postcode)
    if p: return p[0]
    else: return None

In [90]:
# test the function on the postcode set
test_output = set()
for i in postcode_output:
    j = audit_postcode(i)
    test_output.add(j)
print test_output

set(['94213', '94104', '94404', '94401', '94402', '94403', '94563', '94560', '94118', '94611', '94610', '94613', '94612', '94112', '94113', '94110', '94111', '94116', '94618', '94114', '94115', '90214', '94128', '94127', '94579', '94124', '94123', '94122', '94121', '94129', '94577', '94606', '94607', '94605', '94602', '94603', '94601', '94044', '94801', '94804', '94805', '94608', '94609', '94965', '94964', '94130', '94131', '94132', '94133', '94134', '94038', '94970', '94030', '94507', '94501', '94037', '94502', '94587', '94580', '14123', '94303', '94301', '94901', '95498', '94904', '94518', '94519', '94027', '94025', '94596', '94597', '94595', '94598', '94619', '94117', '94017', '94720', '94015', '94014', '94013', '94010', '25426', '94019', '94523', '94158', '94066', '95476', '94108', '94080', '94087', '94002', '94530', '94005', '94143', '94164', '94925', '94920', None, '94708', '94621', '94541', '94707', '94545', '94544', '94709', '94546', '94549', '94706', '94705', '94704', '94703',

In [91]:
# test the audit_postcode function on the whole data set
audit_postcode_set = set()

for elem in get_element(sf_data, tags = ("tag",)):
    if elem.get('k') == "addr:postcode":
        postcode = audit_postcode(elem.get('v'))
        audit_postcode_set.add(postcode)
    elem.clear()

            
print audit_postcode_set

set(['94213', '94404', '94401', '94402', '94403', '94563', '94560', '94118', '94611', '94610', '94613', '94612', '94112', '94113', '94110', '94111', '94116', '94618', '94114', '94115', '90214', '94128', '94127', '94579', '94124', '94123', '94122', '94121', '94129', '94577', '94606', '94607', '94605', '94602', '94603', '94601', '94044', '94801', '94804', '94805', '94608', '94609', '94965', '94964', '94130', '94131', '94132', '94133', '94134', '94038', '94970', '94030', '94507', '94501', '94037', '94502', '94587', '94580', '14123', '94518', '94303', '94301', '94901', '95498', '94904', '94143', '94519', '94027', '94025', '94596', '94597', '94595', '94598', '94619', '94117', '94017', '94720', '94015', '94014', '94013', '94010', '25426', '94019', '94523', '94158', '94109', '94621', '95476', '94080', '94087', '94536', '94530', '94005', '94166', '94164', '94925', '94920', None, '94546', '94104', '94541', '94549', '94545', '94544', '94709', '94708', '94707', '94706', '94705', '94704', '94703',

### The audit_postcode function successfully corrected postcodes

## 1.3 Audit phone number

In [92]:
phone_set = set()
for elem in get_element(sf_data, tags = ("tag",)):
    if elem.get('k') == "phone":
        phone_set.add(elem.get('v')) 
    elem.clear()


In [94]:
#pprint.pprint(list(phone_set))  # to save space in the report. I removed this command after testing

### From the above phone number set, we see that phone numbers have many different formats. Some numbers have country code. Some  numbers don't. Some  numbers are missing area code. Additionally, there are some cases with non-numeric characters. Next, I'd like to further explore to see if there are incorrect phone numbers. 


In [95]:
# define a function to remove all special characters in the phone number to clean up the format
def audit_phone(phone):
    return re.sub('\W', '', phone)
        
#Reference: http://stackoverflow.com/questions/5843518/remove-all-special-characters-punctuation-and-spaces-from-string


In [96]:
audit_phone_set = set([audit_phone(i) for i in phone_set])

In [97]:
for i in audit_phone_set:
    if len(i)== 10:
        pass
    elif len(i)==11:
        pass
    else: print i
        

153581220
415242960
yes
4159291183or
650368384
6667011
6677005
httpwwwpastapastacocom
15108951311or18007867783
8852222
49
fire


### From the audit_phone_set, we see there are only a few numbers of phone numbers are incorrect. Ignore these entries during creating the JSON data file. The next step is to audit phone numbers to a uniform format.

In [98]:
# define a function to create correct phone numbers in a standard format
def standard_phone(phone):
    phone = audit_phone(phone)
    if len(phone)== 10:
        area_code= phone[0:3]
        middle_three = phone[3:6]
        last_four = phone[6:]
        correct_phone = '+1 ' + '(' + area_code + ')' + middle_three + '-' + last_four
    elif len(phone) ==11:
        area_code= phone[1:4]
        middle_three = phone[4:7]
        last_four = phone[7:]
        correct_phone = '+1 ' + '(' + area_code + ')' + middle_three + '-' + last_four
    else: correct_phone = None
    return correct_phone


In [99]:
correct_phone_set = set([standard_phone(i) for i in phone_set])

In [102]:
#pprint.pprint(list(correct_phone_set))  # to save space in the report. I removed this command after testing

### The above standard_phone function successfully corrected the format of phone numbers

## 1.4 Audit street types

In [103]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

In [104]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        street_types[street_type]+=1


In [105]:
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


In [109]:
def audit_street(osmfile):
    
    street_types = defaultdict(int)
    
    for elem in get_element(osmfile, tags = ("node", "way")):
        for tag in elem.iter("tag"):
            if is_street_name(tag):
                audit_street_type(street_types, tag.attrib['v'])
        
        elem.clear()
      
    '''
    context = ET.iterparse(osm_file, events=("start",))
    context = iter(context)
    event, root = context.next()
    for event, elem in context:
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
        
        root.clear()
    osm_file.close()
    
    '''
    
    return street_types 


In [110]:
street_types = audit_street(sf_data)

#Reference: http://stackoverflow.com/questions/12160418/why-is-lxml-etree-iterparse-eating-up-all-my-memory

In [112]:
#pprint.pprint(dict(street_types)) # to save space in the report. I removed this command after testing

### There are many different types of street. Some values are actually not street types. Since such cases are very rare in the dataset, I simply ignore the street types that appear less than 10 times in the dataset when creating the JSON file for database. Audit the rest of street types to a list of standard street types

In [113]:
# create a list of common street types which have more than 10 counts
common_street_types = []
for key, value in street_types.items():
    if value >=10: common_street_types.append(key)

common_street_types.sort()
print "Number of common street types:", len(common_street_types)
pprint.pprint(list(common_street_types))

Number of common street types: 38
['Alameda',
 'Alley',
 'Ave',
 'Ave.',
 'Avenue',
 'Blvd',
 'Boulevard',
 'Bridgeway',
 'Broadway',
 'Center',
 'Circle',
 'Court',
 'Dr',
 'Drive',
 'Embarcadero',
 'Gardens',
 'H',
 'Highway',
 'Lane',
 'Las',
 'Loop',
 'Mason',
 'Ora',
 'Parkway',
 'Path',
 'Place',
 'Plaza',
 'Plz',
 'Rd',
 'Real',
 'Road',
 'Square',
 'St',
 'Street',
 'Terrace',
 'Walk',
 'Way',
 'avenue']


### I found that there are unusual street types in the list, such as "Alameda", "H", ''Gardens'', "Las", "Ora" and "Square". In order to confirm if they are real street types unique to the San Francisco area, I'd like to pull out the full street name of unusual street types. Given the small number of unusual street types, I can examine them manually.  

In [115]:
unusual_street_types = ["Alameda", "H", "Gardens", "Las", "Ora", "Square"]
unusual_street_names = set()
for elem in get_element(sf_data, tags= ("tag")):
    if elem.get('k') == "addr:street":
        v = elem.get('v')
        m = street_type_re.search(v)
        if m:
            if m.group() in unusual_street_types:
                unusual_street_names.add(elem.get('v')) 
    elem.clear()


In [116]:
pprint.pprint(list(unusual_street_names))

['Avenue H',
 'Eureka Square',
 'Jack London Square',
 'The Alameda',
 'Union Square',
 'Avenue Del Ora',
 'Alameda De Las',
 'Shattuck Square',
 'West Mall Square',
 'Wildwood Gardens',
 'Alameda',
 'Orinda Theatre Square',
 'Berkeley Square']


### By googling the above unusual street names, I found that some of the streets actually exist. However, some are landmarks, not street names. Therefore, I decided to keep the existing street names and remove landmarks, including "Gardens", "Sqaure" and "Alameda" from common street type list.

In [117]:
remove_street = ["Gardens","Square","Alameda"]
for i in remove_street:
    common_street_types.remove(i)

In [118]:
print "Common street types:", len(common_street_types)
pprint.pprint(list(common_street_types))

Common street types: 35
['Alley',
 'Ave',
 'Ave.',
 'Avenue',
 'Blvd',
 'Boulevard',
 'Bridgeway',
 'Broadway',
 'Center',
 'Circle',
 'Court',
 'Dr',
 'Drive',
 'Embarcadero',
 'H',
 'Highway',
 'Lane',
 'Las',
 'Loop',
 'Mason',
 'Ora',
 'Parkway',
 'Path',
 'Place',
 'Plaza',
 'Plz',
 'Rd',
 'Real',
 'Road',
 'St',
 'Street',
 'Terrace',
 'Walk',
 'Way',
 'avenue']


### In the above list of common street types, there is inconsistency for a given street type. For example, "Avenue" has the three forms -  "Ave", "Avenue" and "avenue". The next step is to audit the street types and create a list of standard common street types.

In [119]:
# create a map to correct inconsistent street types
mapping = { 
            "Ave": 'Avenue',
            "Ave.": 'Avenue',
           "avenue": 'Avenue',
            "Blvd": "Boulevard",
           "Dr": "Drive",
           "Plz": "Plaza",
           'Rd': 'Road',
           "St": "Street"
            }

expected = common_street_types

In [120]:
#based on the most common street types, create a standard list street types
for key, value in mapping.items():
    expected.remove(key)

In [121]:
print "Number of expected street types", len(expected)
print expected

Number of expected street types 27
['Alley', 'Avenue', 'Boulevard', 'Bridgeway', 'Broadway', 'Center', 'Circle', 'Court', 'Drive', 'Embarcadero', 'H', 'Highway', 'Lane', 'Las', 'Loop', 'Mason', 'Ora', 'Parkway', 'Path', 'Place', 'Plaza', 'Real', 'Road', 'Street', 'Terrace', 'Walk', 'Way']


### The above list of expected street types are the final list of street types that will show up in the audited JSON file 

In [122]:
# fix the street names according to the above map. Ignore the rest of uncommon street names when importing to JSON file. 

def update_name(name, mapping, expected):
    words = name.split()
    st_type = words[-1]
    if st_type in mapping:
        return name.replace(st_type, mapping[st_type])
    elif st_type in expected:
        return name
    else: return None

In [123]:
street_types.keys() # Use this list of street types to test the update_name function

['Rd',
 'Cut',
 '3658',
 'broadway',
 'Alameda',
 'Powell',
 'Stairway',
 'Marina',
 'Path',
 'Building',
 'Loma',
 'I-580',
 '4.5',
 'D',
 'H',
 'Bay',
 'Steps',
 'Wharf',
 'Alley',
 'Folsom',
 'Market/Castro',
 'Broadway',
 'square',
 'West',
 'Cres',
 'Mason',
 'street',
 '3.2',
 'Dr.',
 'Circle',
 'East',
 'Pl',
 'Palms',
 'avenue',
 'Park',
 'Plz',
 '3',
 'Cumbre',
 'I-580)',
 'Ferlinghetti',
 '15th',
 'Ora',
 'King',
 'Ave.',
 'Bradshaw',
 'Plaza',
 'Mall',
 'Place',
 'Rock',
 '100',
 'Blvd,',
 'Loop',
 'Blvd.',
 '39',
 'Square',
 'Lugano',
 'Boulevard',
 'Bridgeway',
 'St.',
 'Rhein',
 'Macdonald',
 'Pulgas',
 'Telegraph',
 'Way',
 '155',
 'Lindbergh',
 'Columbus',
 'North',
 'Trail',
 'Ness',
 'Spencer',
 '730',
 '2',
 'California',
 'Wedemeyer',
 'Post',
 'Lane',
 'B',
 'Center',
 'Bluxome',
 'Arguello',
 'St',
 'Via',
 'Ashfield',
 'Market/Noe',
 'Parkway',
 'Southgate',
 'Road',
 '15',
 'Int',
 'Walk',
 'Sobrante',
 'Ave',
 'parkway',
 'Avenue',
 'Market',
 'Ct',
 'Oakridge'

In [124]:
# test the function of update_name on street types 
for i in street_types.keys():
    audit_list = mapping.keys()
    name = update_name(i, mapping, expected)
    print "Before updating:", i, "==>", "after updating:", name

Before updating: Rd ==> after updating: Road
Before updating: Cut ==> after updating: None
Before updating: 3658 ==> after updating: None
Before updating: broadway ==> after updating: None
Before updating: Alameda ==> after updating: None
Before updating: Powell ==> after updating: None
Before updating: Stairway ==> after updating: None
Before updating: Marina ==> after updating: None
Before updating: Path ==> after updating: Path
Before updating: Building ==> after updating: None
Before updating: Loma ==> after updating: None
Before updating: I-580 ==> after updating: None
Before updating: 4.5 ==> after updating: None
Before updating: D ==> after updating: None
Before updating: H ==> after updating: H
Before updating: Bay ==> after updating: None
Before updating: Steps ==> after updating: None
Before updating: Wharf ==> after updating: None
Before updating: Alley ==> after updating: Alley
Before updating: Folsom ==> after updating: None
Before updating: Market/Castro ==> after updatin

### The above update_name function successfully corrected street types

## So far, I have identified three problems with the dataset and wrote functions to correct them. The three problems encountered are:
### 1. Postcodes have different formats and invalid postcode
### 2. Phone numbers have different formats and invalide numbers
### 3. Street types have different formats and invalid street types


## 1.5 Parse audited data into JSON file and import it into MongoDB

In [132]:
import codecs
import json

In [126]:
expected # standard street types

['Alley',
 'Avenue',
 'Boulevard',
 'Bridgeway',
 'Broadway',
 'Center',
 'Circle',
 'Court',
 'Drive',
 'Embarcadero',
 'H',
 'Highway',
 'Lane',
 'Las',
 'Loop',
 'Mason',
 'Ora',
 'Parkway',
 'Path',
 'Place',
 'Plaza',
 'Real',
 'Road',
 'Street',
 'Terrace',
 'Walk',
 'Way']

In [127]:
mapping # abbreviations of street types that need to be improved

{'Ave': 'Avenue',
 'Ave.': 'Avenue',
 'Blvd': 'Boulevard',
 'Dr': 'Drive',
 'Plz': 'Plaza',
 'Rd': 'Road',
 'St': 'Street',
 'avenue': 'Avenue'}

In [128]:
CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

In [129]:
def shape_element(element):
    node = {}
    if element.tag == "node" or element.tag == "way" :
        node['id']= element.get('id')
        node['type'] = element.tag
        node['visible'] = element.get('visible')
        node['created'] ={}
        for key in CREATED:
            node['created'][key] = element.get(key)
        if element.get('lat') != None and element.get('lon') != None:
            node['pos'] = [ float(element.get('lat')), float(element.get('lon'))]
       
        address = {}
        node_refs = []
        for child in element:
            if child.tag =="tag":
                k = child.get('k')
                
                val = child.get('v')
                if k and val:
                    
                    if re.search('^addr:', k):
                       
                        key = re.findall('^addr:(.+$)', k)[0]
                        if key == "postcode":
                            val = audit_postcode(val)
                        elif key == "street":
                            val = update_name(val, mapping, expected)
                        else: pass
                        address[key]= val

                    elif k == "phone":
                        val = standard_phone(val)
                        node[k] = val
                    else: 
                        node[k]= val
            
            elif child.tag == 'nd':
                node_refs.append(child.get('ref'))
        if address:
            node["address"] = address
        if node_refs:
            node["node_refs"] = node_refs
        return node
    else:
        return None


In [130]:
import time

def process_map(file_in, pretty = False):
    t0 = time.time()
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        
        for element in get_element(file_in, tags=("node", "way")):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
            
    t1 = time.time()

    total = t1-t0
    print "total process_map time:", total
    return file_out
# The functions process_map and get_element are efficient to process large data file     

In [133]:
process_map("san-francisco_california.osm")

total process_map time: 247.028000116


'san-francisco_california.osm.json'

### The improved get_element() and process_map() functions are able to process the original file (about 1GB) efficiently in time and memory. 

In [134]:
#process_map("sample_sf.osm")

## Import into MongoDB. Import the .json file through command line as follow

#### > net start MongoDB (in cmd prompt under Administrator role)
#### > mongoimport --db OpenStreetMap --collection SanFrancisco_Yield --file C:\Users\...\san-francisco_california.osm.json

In [136]:
from pymongo import MongoClient
from pprint import pprint
import pymongo

client=MongoClient("localhost", 27017)
db = client.OpenStreetMap


In [137]:
coll = db.SanFrancisco_Yield

### san-francisco_california.osm  file size: 929MB
### san-francisco_california.osm.json   file size: 1.16 GB

# 2 Data overview

### Explore the document fields in the collection

In [139]:
# the goal to check out the format of address information
count =0
j=0
for i in coll.find():
    if count ==3:
        break
    elif "address" in i and "phone" in i:
        pprint(i)
        count +=1
    j+=1

print j

{u'_id': ObjectId('56e4d2b070997f97f1aabd43'),
 u'address': {u'housenumber': u'1601',
              u'postcode': u'94117',
              u'street': u'Haight Street'},
 u'amenity': u'bar',
 u'created': {u'changeset': u'15927535',
              u'timestamp': u'2013-04-30T23:45:40Z',
              u'uid': u'1405729',
              u'user': u'jabel119',
              u'version': u'6'},
 u'id': u'61676294',
 u'name': u"Hobson's Choice",
 u'phone': u'+1 (415)621-5859',
 u'pos': [37.7696028, -122.4487897],
 u'type': u'node',
 u'visible': None,
 u'wifi': u'no'}
{u'_id': ObjectId('56e4d2b070997f97f1aabd48'),
 u'address': {u'city': u'San Francisco',
              u'housenumber': u'1390',
              u'postcode': u'94102',
              u'state': u'CA',
              u'street': u'Market Street'},
 u'amenity': u'post_office',
 u'created': {u'changeset': u'22611423',
              u'timestamp': u'2014-05-29T04:29:32Z',
              u'uid': u'501715',
              u'user': u'rkuris',
           

### Number of total documents in SanFrancisco collection

In [141]:
coll.count()
# The count of documents in collection is equal to the number of "node" and "way" in the .osm file. 
# This demonstrated that all entries are imported into the database

4971220

### Number of nodes

In [142]:
coll.find({"type": "node"}).count()

4466886

### Number of ways

In [143]:
coll.find({"type": "way"}).count()

504216

### Number of unique users that contributed to the dataset

In [144]:
total_unique_user = coll.aggregate([
        {"$group": {"_id": "$created.user", "user_count": {"$sum": 1}}},
        {"$group": {"_id": "total unique users", "count": {"$sum": 1}}},
    ])

In [145]:
print "Number of unique users:" , list(total_unique_user)

Number of unique users: [{u'count': 2175, u'_id': u'total unique users'}]


### Top contributing users

In [146]:
top10_user = coll.aggregate([
        {"$group": {"_id": "$created.user", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 10}
    ])

In [147]:
list(top10_user) # the top 10 contributing users

[{u'_id': u'ediyes', u'count': 944207},
 {u'_id': u'Luis36995', u'count': 720327},
 {u'_id': u'Rub21', u'count': 410652},
 {u'_id': u'RichRico', u'count': 223260},
 {u'_id': u'calfarome', u'count': 184244},
 {u'_id': u'oldtopos', u'count': 168200},
 {u'_id': u'KindredCoda', u'count': 143806},
 {u'_id': u'karitotp', u'count': 132836},
 {u'_id': u'samely', u'count': 123694},
 {u'_id': u'abel801', u'count': 108516}]

### Amenity summary

In [148]:
amenity =  coll.aggregate( [
        {"$group": {"_id": "$amenity", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 10}
        ])


In [149]:
print "top 10 amenities:"
list(amenity)

top 10 amenities:


[{u'_id': None, u'count': 4953455},
 {u'_id': u'parking', u'count': 3987},
 {u'_id': u'restaurant', u'count': 2650},
 {u'_id': u'school', u'count': 1304},
 {u'_id': u'place_of_worship', u'count': 1119},
 {u'_id': u'bench', u'count': 964},
 {u'_id': u'cafe', u'count': 834},
 {u'_id': u'post_box', u'count': 646},
 {u'_id': u'fast_food', u'count': 584},
 {u'_id': u'bicycle_parking', u'count': 480}]

### Number of records based on postcode

In [150]:
postcode =  coll.aggregate( [
        {"$group": {"_id": "$address.postcode", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 10}
        ])


In [151]:
print "Top 10 postcode area with most records:"
list(postcode)

Top 10 postcode area with most records:


[{u'_id': None, u'count': 4952162},
 {u'_id': u'94122', u'count': 4580},
 {u'_id': u'94611', u'count': 2982},
 {u'_id': u'94116', u'count': 2022},
 {u'_id': u'94610', u'count': 1349},
 {u'_id': u'94133', u'count': 1057},
 {u'_id': u'94117', u'count': 993},
 {u'_id': u'94127', u'count': 597},
 {u'_id': u'94103', u'count': 505},
 {u'_id': u'94109', u'count': 440}]

### Religions in the area

In [74]:
'''
# look up the information of document with "place of worship"
count =0
j=0
for i in coll.find():
    if count ==3:
        break
    else:
        if "amenity" not in i: continue
        else:
            v = i['amenity']
            if v == "place_of_worship":
                pprint(i)
                count +=1
'''


'\n# look up the information of document with "place of worship"\ncount =0\nj=0\nfor i in coll.find():\n    if count ==3:\n        break\n    else:\n        if "amenity" not in i: continue\n        else:\n            v = i[\'amenity\']\n            if v == "place_of_worship":\n                pprint(i)\n                count +=1\n'

In [152]:
place_of_worship =  coll.aggregate( [
        {"$group": {"_id": "$religion", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 10}
        ])

In [153]:
print "Top 10 religions in the area:"
list(place_of_worship)

Top 10 religions in the area:


[{u'_id': None, u'count': 4970114},
 {u'_id': u'christian', u'count': 1032},
 {u'_id': u'buddhist', u'count': 31},
 {u'_id': u'jewish', u'count': 20},
 {u'_id': u'muslim', u'count': 8},
 {u'_id': u'taoist', u'count': 3},
 {u'_id': u'unitarian', u'count': 2},
 {u'_id': u'scientologist', u'count': 2},
 {u'_id': u'unitarian_universalist', u'count': 2},
 {u'_id': u'shinto', u'count': 1}]

### Cuisine in the area

In [154]:
cuisine =  coll.aggregate( [
        {"$group": {"_id": "$cuisine", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 10}
        ])


In [155]:
print "Top 10 cuisine in San Francisco:"
list(cuisine)

Top 10 cuisine in San Francisco:


[{u'_id': None, u'count': 4968789},
 {u'_id': u'mexican', u'count': 244},
 {u'_id': u'coffee_shop', u'count': 230},
 {u'_id': u'burger', u'count': 191},
 {u'_id': u'pizza', u'count': 191},
 {u'_id': u'chinese', u'count': 156},
 {u'_id': u'sandwich', u'count': 130},
 {u'_id': u'japanese', u'count': 122},
 {u'_id': u'italian', u'count': 120},
 {u'_id': u'american', u'count': 110}]

### San Francisco is famous for its diverse culture, which is reflected in the popularity of international cuisines. 

### Explore the most popular cuisine in each postcode area of San Francisco

In [161]:
cuisine_of_area =  coll.aggregate( [
        {"$match": {"cuisine": {"$exists":1},
                    "address.postcode": {"$exists": 1}}},
        {"$group": {"_id": { "postcode": "$address.postcode",
                             "cuisine": "$cuisine"},
                    "cuisine_count": {"$sum": 1}}},
        {"$match": {"cuisine_count": {"$gt": 2}}},
        
        {"$sort": {"cuisine_count": -1 }},
        {"$group": {"_id": "$_id.postcode",
                    "cuisine_of_area": {"$push": {
                                                "cuisine": "$_id.cuisine",
                                                "count": "$cuisine_count"
                    }},
                    "total_count":  {"$sum":1}
                   }},
        
        {"$sort": {"total_count":-1}},
        {"$limit": 5}
        ])

list(cuisine_of_area)

#Reference: http://stackoverflow.com/questions/22932364/mongodb-group-values-by-multiple-fields


[{u'_id': u'94122',
  u'cuisine_of_area': [{u'count': 13, u'cuisine': u'chinese'},
   {u'count': 10, u'cuisine': u'coffee_shop'},
   {u'count': 10, u'cuisine': u'japanese'},
   {u'count': 8, u'cuisine': u'vietnamese'},
   {u'count': 5, u'cuisine': u'thai'},
   {u'count': 5, u'cuisine': u'mexican'},
   {u'count': 4, u'cuisine': u'asian'},
   {u'count': 4, u'cuisine': u'indian'},
   {u'count': 3, u'cuisine': u'pizza'},
   {u'count': 3, u'cuisine': u'korean'},
   {u'count': 3, u'cuisine': u'sandwich'},
   {u'count': 3, u'cuisine': u'dim_sum'}],
  u'total_count': 12},
 {u'_id': u'94063',
  u'cuisine_of_area': [{u'count': 12, u'cuisine': u'mexican'},
   {u'count': 12, u'cuisine': u'sandwich'},
   {u'count': 8, u'cuisine': u'burger'},
   {u'count': 7, u'cuisine': u'chinese'},
   {u'count': 6, u'cuisine': u'japanese'},
   {u'count': 6, u'cuisine': u'american'},
   {u'count': 4, u'cuisine': u'italian'},
   {u'count': 4, u'cuisine': u'thai'},
   {u'count': 3, u'cuisine': u'pizza'},
   {u'count'

### Based on the above data, I find that different areas have different popular food. This is probably due to the different population in each district. Ideally, I'd like to check the top 3 popular food in postcode area with a large number of restaurants. However, I cannot figure out an easy way to look at this. 

# 3 Other ideas about the dataset

### I noticed that some documents have phone number under "contact:phone" field. However, some have "phone" as a key in the dictionary, which is the field that has been audited in this report. In order to keep contact information clean and consistent, I propose the following further cleanup:
**First, we should look thoroughly to find out all formats of contact information. Second, we can then audit such contact information, including email, phone, website, etc. Third, we create a dictionary for all types of contact information for each entry in the json file. **
**Potential problem: There are two kinds of contact information that need to be audited. The first type is in "contact:phone" format, which is easy to audit using regular expression to extract the information like what I did in auditing street. The second type does not have "contact:". Therefore each type of contact information needs to be found individually. However, there may be other types of contact types not mentioned above. We may have the chance to lose such information.  **

In [165]:
# show example of "contact:phone"
count=0

for i in coll.find():
    if count ==2 :
        break
    else:
        if "contact:phone" in i:
            pprint(i)
            count +=1


{u'_id': ObjectId('56e4d2b070997f97f1aabd3a'),
 u'address': {u'housenumber': u'547', u'street': u'Haight Street'},
 u'amenity': u'pub',
 u'contact:email': u'info@toronado.com',
 u'contact:fax': u'415 621 0322',
 u'contact:myspace': u'toronadosf',
 u'contact:phone': u'415 863 2276',
 u'contact:website': u'http://www.toronado.com/',
 u'created': {u'changeset': u'36033983',
              u'timestamp': u'2015-12-18T19:11:20Z',
              u'uid': u'251543',
              u'user': u'ChrissW-R1',
              u'version': u'6'},
 u'id': u'61671096',
 u'name': u'Toronado',
 u'opening_hours': u'11:00-02:00',
 u'pos': [37.7719291, -122.4311197],
 u'type': u'node',
 u'visible': None}
{u'_id': ObjectId('56e4d2b570997f97f1acae46'),
 u'amenity': u'cafe',
 u'contact:phone': u'+1-510-849-0211',
 u'contact:website': u'http://musicaloffering.com/cafe/',
 u'cost:coffee': u'$3.01',
 u'created': {u'changeset': u'37351932',
              u'timestamp': u'2016-02-21T18:21:22Z',
              u'uid': u'1835

# Conclusion

### Through an initial examination of the dataset, I found three problems with the dataset and created different functions to clean the data. It is obvious that thare are more fields that need to be audited. After importing the json format of the dataset, I did several queries to examine the data in San Francisco. 
### There are several power useser that contributed to the openstreetmap in more than 100,000 entries. Christian is the doninant religion in the city. However, the food culture is very diverse. Many different types of international foods are popular. This is a unique feature of the diverse culture of San Francisco. Through futher analysis, I show the most popular cuisine in each specific postcode area.
### Additional data wrangling can be done to further improve the data quality. For instance, we can audit the contact information more carefully and create a dictionary of contact for each address. Overall, there are many missing entries and inconsistency in the data, which need to be further cleaned.  
